In [1]:
from glob import glob
import matplotlib.pyplot as plt
import fileinput
import collections
import re
from gensim.models import KeyedVectors, WordEmbeddingSimilarityIndex, TfidfModel
from gensim.similarities import MatrixSimilarity, SparseTermSimilarityMatrix, SoftCosineSimilarity
from gensim.corpora import Dictionary
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

partie = [
    'KO',
    'Konfederacja',
    'Lewica',
    'niez',
    'PiS',
    'PSL-Kukiz15'
]

In [2]:
stopwords = open("polish.stopwords.txt", encoding='utf-8').read().splitlines()

documents = []

for p in partie:
    wypowiedzi_partii = ''
    for posl in glob('partie/' + p + '/*'):
        wypowiedzi = glob(posl + '/*')
        with fileinput.input(wypowiedzi, openhook=fileinput.hook_encoded("windows-1250")) as wyp:
            for line in wyp:
                wypowiedzi_partii += line

    wypowiedzi_partii = wypowiedzi_partii.replace('\n', ' ')
    wypowiedzi_partii = re.sub('[,.:;!?()]', '', wypowiedzi_partii)

    documents.append(
        [word for word in wypowiedzi_partii.lower().split() if word not in stopwords])

In [5]:
wv = KeyedVectors.load("word2vec_100_3_polish.bin")
dictionary = Dictionary(documents)
tfidf = TfidfModel(dictionary=dictionary)

In [4]:
similarity_index = WordEmbeddingSimilarityIndex(wv)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

In [5]:
index = SoftCosineSimilarity(tfidf[[dictionary.doc2bow(document) for document in documents]], similarity_matrix)

index.save('soft_cosine.index')

In [3]:
index = SoftCosineSimilarity.load('soft_cosine.index')

In [63]:
try:
    # query = input("Query: ").lower().split()
    query = "musimy bronić kościołów przed inwazją lgbt".lower().split()
    bow = dictionary.doc2bow(query, return_missing=True)
    query = tfidf[bow[0]]
    print("Słowa nierozpoznane: ", bow[1])
    similarities = index[query]
    for score, partia in sorted(zip(similarities, partie), key= lambda t: t[0], reverse=True):
        print('{:.3f} : {}'.format(score, partia))
except Exception as e:
    print(e)

Słowa nierozpoznane:  {'przed': 1}
0.417 : Konfederacja
0.106 : KO
0.105 : Lewica
0.102 : PiS
0.100 : niez
0.087 : PSL-Kukiz15
